In [24]:
%pip install "pymongo[srv]" python-dotenv


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from dotenv import load_dotenv
import os
import pandas as pd

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv()

True

In [26]:
url = os.getenv("MONGODB_CONNECTION_URL")

DATA_DIR = 'data/'

# url

In [27]:
client = MongoClient(url, server_api=ServerApi('1'))

client.admin.command('ping')
print("Pinged your deployment successfully!")

database = client['temps']
collection = database['obe-temperatures']

Pinged your deployment successfully!


This is how to upload _one_ document

In [28]:
# result = collection.insert_one({
#     "temperature": 25.5,
#     "timestamp": "2021-09-01T12:00:00Z",
#     "city": "Uppsala"
# })

# print(result.acknowledged)

Edit this function so that it uploads data to the mongodb collection instead!

In [31]:
def insert_temps_from_csv(file_path, city_name):
    # Find header row
    header_row = -1
    with open(file_path, 'r') as fp:
        for i, line in enumerate(fp):
            if line.startswith('Datum;Tid (UTC)'):
                header_row = i
                break

    if (header_row < 0):
        print("Could not find header in file!")
        return

    # Upload data to mongodb
    data = pd.read_csv(file_path, sep=";", skiprows=header_row)
    data['DatumTid'] = data['Datum'] + ' ' + data['Tid (UTC)']

    data = data[['DatumTid', 'Lufttemperatur']]
    data.columns = ['timestamp', 'temperature']
    data['city'] = city_name

    data_as_dict = data.to_dict('records')

    collection.insert_many(data.to_dict('records'))

In [32]:
insert_temps_from_csv(DATA_DIR + 'temp-uppsala.csv', 'Uppsala')
insert_temps_from_csv(DATA_DIR + 'temp-lulea.csv', 'Luleå')
insert_temps_from_csv(DATA_DIR + 'temp-goteborg.csv', 'Göteborg')

/var/folders/pg/6tvtzc815214nstb4nc45dcw0000gn/T/ipykernel_78001/2798823563.py:15: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=";", skiprows=header_row)


/var/folders/pg/6tvtzc815214nstb4nc45dcw0000gn/T/ipykernel_78001/2798823563.py:15: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=";", skiprows=header_row)


In [33]:
client.close()